In [1]:
!pip install pyod
!pip install --upgrade pyod  # to make sure that the latest version is installed!

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.0/150.0 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyod: filename=pyod-1.0.9-py3-none-any.whl size=184097 sha256=f463beba0c26875d0f95749a5446939d42f1b0fe8fc40f953e7416de380228e9
  Stored in directory: /root/.cache/pip/wheels/83/55/6b/552e083cf5509c0afe808b76cf434f1be284d01a112623bd37
Successfully built pyod
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
import matplotlib.font_manager
import matplotlib.pyplot as plt
state=42

In [3]:

from scipy import stats
import matplotlib.pyplot as plt
from pyod.models.iforest import IForest
from pyod.models.abod import ABOD
from pyod.models.knn import KNN
from pyod.models.iforest import IForest
from pyod.models.cblof import CBLOF
from pyod.models.hbos import HBOS
from pyod.models.ocsvm import OCSVM


In [29]:
class UnivariateAnamolyDetection():
  def __init__(self ):
    #self.input_file = input_file
    # by default the outlier fraction is 0.1 in generate data function
    self.outlier_fraction = 0.2
    self.models = {
     'ABOD'   : ABOD(contamination=self.outlier_fraction),
     'IF': IForest(contamination=self.outlier_fraction,random_state=state),
     'CBLOF':CBLOF(contamination=self.outlier_fraction,check_estimator=False, random_state=state),
     'HBOS': HBOS(contamination=self.outlier_fraction),
     'KNN' :  KNN(contamination=self.outlier_fraction),     
     "OCSVM":OCSVM(kernel='rbf', degree=3, gamma=0.1,nu=0.05,max_iter=-1)
     
}

  def fit_model_2(self, model, data, column , model_name):
      
      df = data.copy()
      data_to_predict = data[column].to_numpy().reshape(-1, 1).astype(float)
      model.fit(data_to_predict)
      # predict raw anomaly score
      scores_pred = model.decision_function(data_to_predict)*-1

      # prediction of a datapoint category outlier or inlier
      y_pred = model.predict(data_to_predict)
      #predictions = model.fit_predict(data_to_predict)
      c_name_prediction = column + "_" + model_name + "_Predictions"
      c_name_score = column + "_" + model_name + "_Score"
      df[c_name_prediction] = y_pred
      df[c_name_score] = scores_pred
      # threshold value to consider a datapoint inlier or outlier
      threshold = stats.scoreatpercentile(scores_pred,100 * self.outlier_fraction)
      return df,threshold

  def get_anamoly(self , data, column , evaluation_model = "IF" ):
    model = self.models[evaluation_model]
    df,threshold = self.fit_model_2(model, df_student,column , evaluation_model)
    return df,threshold

In [7]:
file_name = "student_marks.xlsx"
columns = ['Marks_20', 'Marks_50', 'Marks_100']
df_student = pd.read_excel(file_name)
df_student.columns

Index(['RollNumber', 'Marks_20', 'Marks_50', 'Marks_100'], dtype='object')

In [33]:
uad = UnivariateAnamolyDetection()
df_student_anamoly = df_student.copy(deep = True)
for column in columns:
  print("Running for " , columns)
  df_student_anamoly = uad.get_anamoly(df_student_anamoly,column , "IF")
  #break
df_student_anamoly

Running for  ['Marks_20', 'Marks_50', 'Marks_100']
Running for  ['Marks_20', 'Marks_50', 'Marks_100']
Running for  ['Marks_20', 'Marks_50', 'Marks_100']


(    RollNumber  Marks_20  Marks_50  Marks_100  Marks_100_IF_Predictions  \
 0            1        12         3         71                         0   
 1            2        13        23          5                         1   
 2            3        -8        36         59                         0   
 3            4        12        34         53                         0   
 4            5        14        34         57                         0   
 5            6        12        23         52                         0   
 6            7        22        20         67                         0   
 7            8        16        25         50                         0   
 8            9         8        35         65                         0   
 9           10        10        41         63                         0   
 10          11        13        27         56                         0   
 11          12        11        23         58                         0   
 12         

In [32]:
display(df_student_anamoly)

(    RollNumber  Marks_20  Marks_50  Marks_100  Marks_100_IF_Predictions  \
 0            1        12         3         71                         0   
 1            2        13        23          5                         1   
 2            3        -8        36         59                         0   
 3            4        12        34         53                         0   
 4            5        14        34         57                         0   
 5            6        12        23         52                         0   
 6            7        22        20         67                         0   
 7            8        16        25         50                         0   
 8            9         8        35         65                         0   
 9           10        10        41         63                         0   
 10          11        13        27         56                         0   
 11          12        11        23         58                         0   
 12         

In [14]:
outlier_fraction = 0.1
models = {
     'Angle-based Outlier Detector (ABOD)'   : ABOD(contamination=outlier_fraction),

     'Isolation Forest': IForest(contamination=outlier_fraction,random_state=state),

     'Cluster-based Local Outlier Factor (CBLOF)':CBLOF(contamination=outlier_fraction,check_estimator=False, random_state=state),

     'Histogram-base Outlier Detection (HBOS)': HBOS(contamination=outlier_fraction),

     'K Nearest Neighbors (KNN)' :  KNN(contamination=outlier_fraction),
     
     "Support Vector Machine":OCSVM(kernel='rbf', degree=3, gamma=0.1,nu=0.05,max_iter=-1)
}


models.keys()

dict_keys(['Angle-based Outlier Detector (ABOD)', 'Isolation Forest', 'Cluster-based Local Outlier Factor (CBLOF)', 'Histogram-base Outlier Detection (HBOS)', 'K Nearest Neighbors (KNN)', 'Support Vector Machine'])